# Web scraping

Este notebook é uma demonstração de como extrair dados de votação de algum evento parlamentar da Câmara dos Deputados.

Você pode utulizá-lo para extrair dados das votações que seguem o mesmo padrão html da utilizada nesse exemplo.

### Documentação das bibliotecas utilizadas: 

<a href="https://docs.python-requests.org/en/latest/" title="Requests">Requests</a>

<a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/#" title="BeautifulSoup">BeautifulSoup</a>

<a href="https://pandas.pydata.org/docs/" title="Pandas">Pandas</a>

<a href="https://docs.python.org/3/library/re.html" title="Regular Expressions">Regular Expressions</a>

### Permissões:

Caso use esse algoritmo para raspagem, não utilizem urls que violem o <a href="https://www.camara.leg.br/robots.txt" title="Protocolo de Exclusão de Robôs">Protocolo de Exclusão de Robôs</a>:

- User-Agent: * 
- Disallow: /sileg/prop_lista* 
- Disallow: /internet/sileg/prop_lista* 
- Disallow: /sileg/Prop_lista* 
- Disallow: /internet/sileg/Prop_lista* 
- Disallow: /*arvore-de-apensados 
- Disallow: /proposicoesWeb/prop_arvore_tramitacoes 
- Disallow: /sileg/prop_arvore_tramitacoes 
- Disallow: /internet/deputado/Dep_Lista* 
- Disallow: /transparencia/recursos-humanos/remuneracao/*
- Disallow: /transparencia/recursos-humanos/contratos-terceirizacao* 
- Disallow: /transparencia/recursos-humanos/funcionarios* 

In [1]:
# importando as bibliotecas necessárias
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re # expressões regulares

In [2]:
# Extraindo o html da página
url = 'https://www.camara.leg.br/presenca-comissoes/votacao-portal?reuniao=62583&itemVotacao=9968&ordenacao=Nome'
r = requests.get(url) 
text = r.text 
print(text)





<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8">

  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
  <meta http-equiv="X-UA-Compatible" content="IE=edge" />
  <meta name="apple-itunes-app" content="app-id=1049343067" />
  <meta name="google-play-app" content="app-id=br.leg.camara.infolegmovel" />
  <title>Portal da Câmara dos Deputados</title>
<style type="text/css">@import url(https://www2.camara.leg.br/portal_css/camara_leg_viradadigital/member-cachekey-01563218998c9e36fc22b9a9e975ff88.css);</style><style type="text/css">@import url(https://www2.camara.leg.br/portal_css/camara_leg_viradadigital/portlets-cachekey-996c2c14a4a7f928d077328f2e83bf9d.css);</style><style type="text/css" media="screen">@import url(https://www2.camara.leg.br/portal_css/camara_leg_viradadigital/deprecated-cachekey-bd45b4a65d3ba1a4c60a6c49fd20abab.css);</style><style type="text/css" media="screen">@import url(https://www2.camara.leg.br/portal_css/camara_leg_viradadigital

In [3]:
# crinado o objeto soup
soup = BeautifulSoup(text) 

# visualizando o objeto soup:
print(soup.prettify()) # com identação

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=1049343067" name="apple-itunes-app"/>
  <meta content="app-id=br.leg.camara.infolegmovel" name="google-play-app"/>
  <title>
   Portal da Câmara dos Deputados
  </title>
  <style type="text/css">
   @import url(https://www2.camara.leg.br/portal_css/camara_leg_viradadigital/member-cachekey-01563218998c9e36fc22b9a9e975ff88.css);
  </style>
  <style type="text/css">
   @import url(https://www2.camara.leg.br/portal_css/camara_leg_viradadigital/portlets-cachekey-996c2c14a4a7f928d077328f2e83bf9d.css);
  </style>
  <style media="screen" type="text/css">
   @import url(https://www2.camara.leg.br/portal_css/camara_leg_viradadigital/deprecated-cachekey-bd45b4a65d3ba1a4c60a6c49fd20abab.css);
  </style>
  <style media="screen" type="text/css">
   @import url(https://www2.camara.leg.

Abaixo temos um exemplo de um elemento que contém todas as informações que precisamos por palarmentar.

    <li>
      <span class="nome">Abou Anni</span>
      <span class="nomePartido">(PSL-SP)</span>
      <span class="votou">-votou<span class="voto sim">Sim</span></span>
     </li>

Precisamos navegar até o elemento de cada deputado e extrair as informações relevantes.

In [4]:
array = []
for elemento in soup.find_all('span', attrs={"class":"nome"}):
# for elemento in soup.select('li span.nome'): # usando select e CSS
    
    lista = []
    
    # nome do deputado ou deputada
    lista.append(elemento.parent.contents[1].string) 
    
    
    lista.append(elemento.parent.contents[3].string) # partido e estado

    # voto
    # lista não tem método .string, apenas o elemento nela:
    # lista vazia não tem elemento com index
    if not elemento.parent.select('span.votou span'): # lista vazia não tem método .string:
        lista.append('Ausente') # ausência
    else:
        lista.append(elemento.parent.select('span.votou span')[0].string) # voto

    array.append(lista)

In [5]:
array

[['AJ Albuquerque', '(PP-CE)', 'Ausente'],
 ['Abílio Santana', '(PL-BA)', 'Ausente'],
 ['Abou Anni', '(PSL-SP)', 'Sim'],
 ['Acácio Favacho', '(PROS-AP)', 'Ausente'],
 ['Adolfo Viana', '(PSDB-BA)', 'Ausente'],
 ['Adriana Ventura', '(Novo-SP)', 'Não'],
 ['Adriano do Baldy', '(PP-GO)', 'Ausente'],
 ['Aécio Neves', '(PSDB-MG)', 'Abstenção'],
 ['Aelton Freitas', '(PL-MG)', 'Não'],
 ['Afonso Florence', '(PT-BA)', 'Não'],
 ['Afonso Hamm', '(PP-RS)', 'Ausente'],
 ['Afonso Motta', '(PDT-RS)', 'Não'],
 ['Aguinaldo Ribeiro', '(PP-PB)', 'Ausente'],
 ['Airton Faleiro', '(PT-PA)', 'Não'],
 ['Alan Rick', '(DEM-AC)', 'Sim'],
 ['Alceu Moreira', '(MDB-RS)', 'Sim'],
 ['Alcides Rodrigues', '(Patriota-GO)', 'Não'],
 ['Alê Silva', '(PSL-MG)', 'Sim'],
 ['Alencar S. Braga', '(PT-SP)', 'Não'],
 ['Alessandro Molon', '(PSB-RJ)', 'Não'],
 ['Alex Manente', '(Cidadania-SP)', 'Não'],
 ['Alex Santana', '(PDT-BA)', 'Sim'],
 ['Alexandre Frota', '(PSDB-SP)', 'Não'],
 ['Alexandre Leite', '(DEM-SP)', 'Não'],
 ['Alexandre 

In [6]:
df = pd.DataFrame(array, columns=['nome', 'partido', 'voto'])

df.to_csv('votos_pec135.csv', index=False)